<a href="https://colab.research.google.com/github/2003UJAN/Phising-Website/blob/main/url_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tldextract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 2.2 MB/s eta 0:00:00


In [3]:
pip install requests

In [4]:
pip install dnspython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 3.7 MB/s eta 0:00:00


In [5]:
pip install python-whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103246 sha256=c71b146dd64ac6ae84c168fbacb04f89124f1b38a4b1164af5d0bd4ee3d13a82
  Stored in directory: /root/.cache/pip/wheels/10/f1/87/145023b9a206e2e948be6480c61ef3fd3dbb81ef11b6977782
Successfully built python-whois


In [6]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from urllib.parse import urlparse,parse_qs
import csv
from google.colab import files
import requests
from bs4 import BeautifulSoup
import tldextract
import socket
import re
import requests
import time
import dns.resolver
import whois
import ssl
from datetime import datetime

In [7]:
#loading the dataset
data = pd.read_csv("dataset_full.csv")

In [8]:
#data pre-processing
data = data.drop(data.columns[0], axis=1)
data = shuffle(data)
threshold = 0.9 * len(data)
data = data.dropna(thresh=threshold, axis=1)

In [9]:
first_row = data.iloc[0]

In [10]:
print(first_row)

qty_hyphen_url          0.0
qty_underline_url       0.0
qty_slash_url           0.0
qty_questionmark_url    0.0
qty_equal_url           0.0
                       ... 
qty_redirects          -1.0
url_google_index        0.0
domain_google_index     0.0
url_shortened           0.0
phishing                0.0
Name: 21053, Length: 111, dtype: float64


In [11]:
urls = [
    'https://www.nitk.ac.in/',
    'https://www.meity.gov.in/',
    'https://www.srmist.edu.in/'
]
domain= 'example.com'
ip_address= '8.8.8.8'
hostname= 'www.example.com'

In [12]:
features={}

In [13]:
def extract_features_from_url(url):
    parsed_url = urlparse(url)
    url_path = parsed_url.path

    features = {
        "dot": url.count('.'),
        "hyphen": url.count('-'),
        "underscore": url.count('_'),
        "slash": url.count('/'),
        "question_mark": url.count('?'),
        "equal": url.count('='),
        "at": url.count('@'),
        "ampersand": url.count('&'),
        "exclamation": url.count('!'),
        "space": url.count(' '),
        "tilde": url.count('~'),
        "comma": url.count(','),
        "plus": url.count('+'),
        "asterisk": url.count('*'),
        "hashtag": url.count('#'),
        "dollar": url.count('$'),
        "percent": url.count('%'),
    }

    return features

In [14]:
def extract_tld_length(url):
    extracted = tldextract.extract(url)
    tld_length = len(extracted.suffix)
    return tld_length

In [15]:
def extract_url_length(url):
    return len(url)

In [16]:
def extract_domain_features(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    features = {
        "dot_domain": domain.count('.'),
        "hyphen_domain": domain.count('-'),
        "underscore_domain": domain.count('_'),
        "slash_domain": domain.count('/'),
        "questionmark_domain": domain.count('?'),
        "equal_domain": domain.count('='),
        "at_domain": domain.count('@'),
        "and_domain": domain.count('&'),
        "exclamation_domain": domain.count('!'),
        "space_domain": domain.count(' '),
        "tilde_domain": domain.count('~'),
        "comma_domain": domain.count(','),
        "plus_domain": domain.count('+'),
        "asterisk_domain": domain.count('*'),
        "hashtag_domain": domain.count('#'),
        "dollar_domain": domain.count('$'),
        "percent_domain": domain.count('%'),
        "vowels_domain": sum(1 for char in domain if char.lower() in 'aeiou')
    }

    return features

In [17]:
def extract_domain_length(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc  # Extract the domain from the URL

    domain_length = len(domain)
    return domain_length

In [18]:
def extract_domain_ip(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    try:
        ip = socket.gethostbyname(domain)
        ip_length = len(ip)
        is_ip = True if ip.count('.') == 3 else False

        features = {
            "domain_in_ip": domain in ip,
            "is_ip_address": is_ip,
            "ip_length": ip_length,
            "ip": ip
        }
    except socket.gaierror:
        features = {
            "domain_in_ip": False,
            "is_ip_address": False,
            "ip_length": 0,
            "ip": "N/A"
        }

    return features

In [19]:
def extract_server_client_keywords(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc.lower()  # Extract the domain from the URL and convert to lowercase

    contains_server = 'server' in domain
    contains_client = 'client' in domain

    features = {
        "server_in_domain": contains_server,
        "client_in_domain": contains_client
    }

    return features

In [20]:
def extract_directory_features(url):
    parsed_url = urlparse(url)
    path = parsed_url.path  # Extract the path/directory from the URL

    features = {
        "dot_directory": path.count('.'),
        "hyphen_directory": path.count('-'),
        "underscore_directory": path.count('_'),
        "slash_directory": path.count('/'),
        "questionmark_directory": path.count('?'),
        "equal_directory": path.count('='),
        "at_directory": path.count('@'),
        "and_directory": path.count('&'),
        "exclamation_directory": path.count('!'),
        "space_directory": path.count(' '),
        "tilde_directory": path.count('~'),
        "comma_directory": path.count(','),
        "plus_directory": path.count('+'),
        "asterisk_directory": path.count('*'),
        "hashtag_directory": path.count('#'),
        "dollar_directory": path.count('$'),
        "percent_directory": path.count('%')
    }

    return features

In [21]:
def extract_directory_length(url):
    parsed_url = urlparse(url)
    path = parsed_url.path  # Extract the path/directory from the URL

    directory_length = len(path)
    return directory_length

In [22]:
def extract_file_features(url):
    parsed_url = urlparse(url)
    path = parsed_url.path
    file_name = path.split('/')[-1]

    features = {
        "dot_file": file_name.count('.'),
        "hyphen_file": file_name.count('-'),
        "underscore_file": file_name.count('_'),
        "slash_file": file_name.count('/'),
        "questionmark_file": file_name.count('?'),
        "equal_file": file_name.count('='),
        "at_file": file_name.count('@'),
        "and_file": file_name.count('&'),
        "exclamation_file": file_name.count('!'),
        "space_file": file_name.count(' '),
        "tilde_file": file_name.count('~'),
        "comma_file": file_name.count(','),
        "plus_file": file_name.count('+'),
        "asterisk_file": file_name.count('*'),
        "hashtag_file": file_name.count('#'),
        "dollar_file": file_name.count('$'),
        "percent_file": file_name.count('%')
    }

    return features


In [23]:
def extract_file_length(url):
    parsed_url = urlparse(url)
    path = parsed_url.path

    file_name = path.split('/')[-1]

    file_length = len(file_name)
    return file_length

In [24]:
def extract_params_features(url):
    parsed_url = urlparse(url)
    query = parsed_url.query
    parsed_query = parse_qs(query)

    params_str = '&'.join([','.join(v) for v in parsed_query.values()])

    features = {
        "dot_params": params_str.count('.'),
        "hyphen_params": params_str.count('-'),
        "underscore_params": params_str.count('_'),
        "slash_params": params_str.count('/'),
        "questionmark_params": params_str.count('?'),
        "equal_params": params_str.count('='),
        "at_params": params_str.count('@'),
        "and_params": params_str.count('&'),
        "exclamation_params": params_str.count('!'),
        "space_params": params_str.count(' '),
        "tilde_params": params_str.count('~'),
        "comma_params": params_str.count(','),
        "plus_params": params_str.count('+'),
        "asterisk_params": params_str.count('*'),
        "hashtag_params": params_str.count('#'),
        "dollar_params": params_str.count('$'),
        "percent_params": params_str.count('%')
    }

    return features

In [25]:
def extract_params_length(url):
    parsed_url = urlparse(url)
    query = parsed_url.query

    params_length = len(query)
    return params_length

In [26]:
def tld_present_in_params(url):
    parsed_url = urlparse(url)
    query_params = parsed_url.query

    tld = tldextract.extract(url).suffix

    parsed_query = parse_qs(query_params)

    tld_present = any(tld in value for values in parsed_query.values() for value in values)

    return tld_present

In [27]:
def count_params(url):
    parsed_url = urlparse(url)
    query_params = parsed_url.query

    parsed_query = parse_qs(query_params)

    num_params = sum(len(values) for values in parsed_query.values())

    return num_params

In [28]:
def email_in_url(url):
    parsed_url = urlparse(url)
    url_string = parsed_url.geturl()

    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    match = re.search(email_pattern, url_string)

    return bool(match)


In [29]:
def time_response(url):
  parsed_url = urlparse(url)
  domain = parsed_url.netloc
  start_time = time.time()
  response = requests.get(url)
  end_time = time.time()
  response_time = end_time - start_time
  return response_time

In [30]:
def has_spf_record(domain):
    try:
        answers = dns.resolver.resolve(domain, 'TXT')
        for rdata in answers:
            if 'v=spf1' in rdata.to_text():
                return True
        return False
    except dns.resolver.NoAnswer:
        return False
    except dns.resolver.NXDOMAIN:
        return False
    except dns.resolver.NoNameservers:
        return False

In [31]:
def get_asn_info(ip_address):
    # Replace 'YOUR_TOKEN' with your IPinfo API token
    token = 'YOUR_TOKEN'
    url = f'https://ipinfo.io/{ip_address}/json?token={token}'

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'asn' in data:
                return data['asn']
            else:
                return "ASN information not found."
        else:
            return "Request failed."
    except requests.RequestException as e:
        return f"Request error: {e}"

In [32]:
def get_domain_dates(domain_name):
    try:
        domain = whois.whois(domain_name)

        # Extract creation and expiration dates
        creation_date = domain.creation_date
        expiration_date = domain.expiration_date

        # Calculate time differences
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]

        # Calculate days until expiration
        if creation_date and expiration_date:
            current_time = datetime.now()
            activation_time = (current_time - creation_date).days
            expiration_time = (expiration_date - current_time).days

            return activation_time, expiration_time
        else:
            return None, None
    except whois.parser.PywhoisError as e:
        return None, None


In [33]:
def get_qty_resolved_ips(domain):
    try:
        ips = socket.getaddrinfo(domain, None)
        return len(set(ip[4][0] for ip in ips))
    except socket.gaierror:
        return 0

def get_qty_nameservers(domain):
    try:
        ns_records = dns.resolver.resolve(domain, 'NS')
        return len(ns_records)
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        return 0

def get_qty_mx_servers(domain):
    try:
        mx_records = dns.resolver.resolve(domain, 'MX')
        return len(mx_records)
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        return 0

In [34]:
def get_dns_ttl(hostname):
   hostname = urlparse(url).hostname  # Extract hostname
   resolver = dns.resolver.Resolver()
   answer = resolver.query(hostname, "A")
   ttl= answer.rrset.ttl
   print("TTL:",ttl)

In [35]:
def get_ssl_certificate_details(url):
    hostname = url.split("//")[-1].split("/")[0]
    context = ssl.create_default_context()
    with socket.create_connection((hostname, 443)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as ssock:
            cert = ssock.getpeercert()
            return cert


In [36]:
def get_redirect_count(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return 0

In [37]:
def check_url_indexing(url):
    try:
        response = requests.get(f"https://www.google.com/search?q=site:{url}")
        return 'did not match any documents' not in response.text
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return False

In [38]:
def check_domain_indexing(domain):
    try:
        response = requests.get(f"https://www.google.com/search?q=site:{domain}")
        return 'did not match any documents' not in response.text
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return False


In [39]:
def is_url_shortened(url):
    length_threshold = 0.6
    original_length = len(url)
    if original_length < 25:
        return False
    else:
        return len(url) <= length_threshold * original_length

In [40]:
for url in urls:
    url_features = extract_features_from_url(url)
    print(f"URL: {url}")
    print(url_features)
    tld_length = extract_tld_length(url)
    print(f"TLD Length: {tld_length}")
    url_length = extract_url_length(url)
    print(f"URL Length: {url_length}")
    domain_features = extract_domain_features(url)
    print(domain_features)
    length = extract_domain_length(url)
    print(f"Domain Length: {length}")
    ip_features = extract_domain_ip(url)
    print(ip_features)
    server_client_features = extract_server_client_keywords(url)
    print(server_client_features)
    directory_features = extract_directory_features(url)
    print(directory_features)
    length = extract_directory_length(url)
    print(f"Directory Length: {length}")
    file_features = extract_file_features(url)
    print(file_features)
    length = extract_file_length(url)
    print(f"File Length: {length}")
    params_features = extract_params_features(url)
    print(params_features)
    length = extract_params_length(url)
    print(f"Parameters Length: {length}")
    tld_in_params = tld_present_in_params(url)
    print(f"TLD Presence in Params: {tld_in_params}")
    param_count = count_params(url)
    print(f"Number of Parameters: {param_count}")
    has_email = email_in_url(url)
    print(f"Email in URL: {has_email}")
    print(f"Response Time: {time_response} seconds")
    has_spf = has_spf_record(domain)
    if has_spf:
      print(f"The domain {domain} has an SPF record.")
    else:
      print(f"The domain {domain} does not have an SPF record.")
    asn_info = get_asn_info(ip_address)
    print(f"The ASN for {ip_address} is: {asn_info}")
    activation_time, expiration_time = get_domain_dates(domain)
    if activation_time is not None and expiration_time is not None:
      print(f"Activation time for {domain}: {activation_time} days")
      print(f"Expiration time for {domain}: {expiration_time} days")
    else:
      print(f"Failed to retrieve activation and expiration times for {domain}")
    qty_ips = get_qty_resolved_ips(domain)
    qty_ns = get_qty_nameservers(domain)
    qty_mx = get_qty_mx_servers(domain)
    print(f"Quantity of resolved IPs for {domain}: {qty_ips}")
    print(f"Quantity of name servers for {domain}: {qty_ns}")
    print(f"Quantity of MX servers for {domain}: {qty_mx}")
    TTL=get_dns_ttl(hostname)
    print("TTL:", TTL)
    cert_details = get_ssl_certificate_details(url)
    print(cert_details)
    redirect_count = get_redirect_count(url)
    print(f"Number of redirects for {url}: {redirect_count}")
    url_indexed = check_url_indexing(url)
    if url_indexed:
      print(f"The URL {url} is indexed on Google.")
    else:
      print(f"The URL {url} is not indexed on Google.")
    domain_indexed = check_domain_indexing(domain)
    if domain_indexed:
      print(f"The domain {domain} is indexed on Google.")
    else:
      print(f"The domain {domain} is not indexed on Google.")
      print(f"Is '{url}' a shortened URL? {is_url_shortened(url)}")

URL: https://www.nitk.ac.in/
{'dot': 3, 'hyphen': 0, 'underscore': 0, 'slash': 3, 'question_mark': 0, 'equal': 0, 'at': 0, 'ampersand': 0, 'exclamation': 0, 'space': 0, 'tilde': 0, 'comma': 0, 'plus': 0, 'asterisk': 0, 'hashtag': 0, 'dollar': 0, 'percent': 0}
TLD Length: 5
URL Length: 23
{'dot_domain': 3, 'hyphen_domain': 0, 'underscore_domain': 0, 'slash_domain': 0, 'questionmark_domain': 0, 'equal_domain': 0, 'at_domain': 0, 'and_domain': 0, 'exclamation_domain': 0, 'space_domain': 0, 'tilde_domain': 0, 'comma_domain': 0, 'plus_domain': 0, 'asterisk_domain': 0, 'hashtag_domain': 0, 'dollar_domain': 0, 'percent_domain': 0, 'vowels_domain': 3}
Domain Length: 14
{'domain_in_ip': False, 'is_ip_address': True, 'ip_length': 13, 'ip': '218.248.46.85'}
{'server_in_domain': False, 'client_in_domain': False}
{'dot_directory': 0, 'hyphen_directory': 0, 'underscore_directory': 0, 'slash_directory': 1, 'questionmark_directory': 0, 'equal_directory': 0, 'at_directory': 0, 'and_directory': 0, 'excl

<ipython-input-34-17e279dc67a1>:4: DeprecationWarning: please use dns.resolver.Resolver.resolve() instead
  answer = resolver.query(hostname, "A")


{'subject': ((('commonName', '*.l1.nitk.ac.in'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '03D54B76DE4EC6C6D723B5A625719D742498', 'notBefore': 'Oct  8 00:06:32 2023 GMT', 'notAfter': 'Jan  6 00:06:31 2024 GMT', 'subjectAltName': (('DNS', '*.l1.nitk.ac.in'), ('DNS', '*.l1.nitk.edu.in'), ('DNS', '*.l2.nitk.ac.in'), ('DNS', '*.l2.nitk.edu.in'), ('DNS', '*.l3.nitk.ac.in'), ('DNS', '*.l3.nitk.edu.in'), ('DNS', '*.l4.nitk.ac.in'), ('DNS', '*.l4.nitk.edu.in'), ('DNS', '*.nitk.ac.in'), ('DNS', '*.nitk.edu.in'), ('DNS', 'nitk.ac.in'), ('DNS', 'nitk.edu.in')), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}
Number of redirects for https://www.nitk.ac.in/: 0
The URL https://www.nitk.ac.in/ is indexed on Google.
The domain example.com is indexed on Google.
URL: https://www.meity.gov.in/
{'dot': 3, 'hyphen': 0, 'underscore': 0, 'slash': 3, 'question_mark': 0, 'equal': 0, '

In [51]:
all_data_to_store = []

for url in urls:
  data_to_store = [
     {
        'URL': url,
        'URL Features':url_features,
        'TLD Length': extract_tld_length,
        'URL Length': url_length,
        'Domain Features': extract_domain_features,
        'Domain Length': extract_domain_length,
        'IP Features':ip_features,
        'Server Client Features':extract_server_client_keywords,
        'Directory Features':extract_directory_features,
        'Directory Length':extract_directory_length,
        'File Features':extract_file_features,
        'File Length':extract_file_length,
        'Parameter Features':extract_params_features,
        'Parameter Length':extract_params_length,
        'TLD Presence in Parameter':tld_in_params,
        'No.of Parameters':param_count,
        'Email in URL':email_in_url,
        'Response Time':time_response,
        'Has spf record':has_spf_record,
        'ASN info':get_asn_info,
        'Activation Time/Expiration Time':get_domain_dates,
        'Number of Resolved IPs':qty_ips,
        'Number of Resolved Name Servers':qty_ns,
       	'Number of MX Servers':qty_mx,
        'Time to live (TTL)':TTL,
        'Valid TLS / SSL Certificate':cert_details,
        'No of Redirects':get_redirect_count,
        'Is URL indexed on Google':check_url_indexing,
        'Is domain indexed on Google':check_domain_indexing,
        'Is URL Shortened': is_url_shortened(url)
     },
   ]
 all_data_to_store.append(url)

# Extract headers from one data entry
headers = list(all_data_to_store[0].keys())

file_name = 'your_file.csv'

with open(file_name, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)

    # Write headers
    writer.writeheader()

    # Write data row by row
    for data in all_data_to_store:
        writer.writerow(data)

print(f"Data saved to '{file_name}'")
  headers = [
        {
              'URL','URL Features','TLD Length','URL Length','Domain Features','Domain Length','IP Features','Server Client Features',
              'Directory Features','Directory Length','File Features','File Length','Parameter Features','Parameter Length','TLD Presence in Parameter',
              'No.of Parameters','Email in URL','Response Time','Has spf record', 'ASN info','Activation Time/Expiration Time','Number of Resolved IPs',
              'Number of Resolved Name Servers','Number of MX Servers','Time to live (TTL)', 'Valid TLS / SSL Certificate','No of Redirects',
              'Is URL indexed on Google','Is domain indexed on Google','Is URL Shortened'
        },
      ]

In [41]:
file_name = 'url_data.csv'

In [54]:
with open('url_data.csv', 'w', newline='') as csvfile:  # Use 'newline=' for consistent CSV format
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    writer.writerows(data_to_store)

print("CSV file saved successfully!")

TypeError: ignored